In [11]:
!pip install lightgbm
!pip install wheel

In [12]:
import numpy as np
import pandas as pd 

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
import time
#from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [13]:
!pip install nlpaug

In [14]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

In [15]:
!pip install textattack

In [16]:
from textattack.augmentation import EmbeddingAugmenter

In [17]:
# from google.colab import drive

# drive.mount('/content/drive')

In [18]:
df = pd.read_csv('label_dataset.csv', lineterminator='\n')

df.head()

,Datetime,Quarter,Likes,NFT,Text,Clean Text,vader,textblob,Polarity (Reviewer 1),Polarity (Reviewer 2)\r
0,2022-06-27,22,2,Mutant Ape Yacht Club,🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#27894...,mutant ape yacht club mayc nft sold eth k,neu,neu,pos,pos\r
1,2023-02-11,31,2,Mutant Ape Yacht Club,🐳1 Mutant Ape Yacht Club bought for Ξ15.377\n\...,mutant ape yacht club bought floor h chg floor...,neu,neu,pos,pos\r
2,2022-12-30,24,0,Mutant Ape Yacht Club,🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#77 so...,mutant ape yacht club mayc nft sold eth k,neu,neu,pos,pos\r
3,2022-12-29,24,0,Mutant Ape Yacht Club,🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#23452...,mutant ape yacht club mayc nft sold eth k,neu,neu,pos,pos\r
4,2022-12-29,24,1,Mutant Ape Yacht Club,Mutant Ape Yacht Club #27908 sold for 17 ETH (...,mutant ape yacht club sold eth nft collection ...,neu,neu,pos,pos\r


In [19]:
df.rename(columns={'Polarity (Reviewer 1)':'label'}, inplace=True)

df.head()

,Datetime,Quarter,Likes,NFT,Text,Clean Text,vader,textblob,label,Polarity (Reviewer 2)\r
0,2022-06-27,22,2,Mutant Ape Yacht Club,🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#27894...,mutant ape yacht club mayc nft sold eth k,neu,neu,pos,pos\r
1,2023-02-11,31,2,Mutant Ape Yacht Club,🐳1 Mutant Ape Yacht Club bought for Ξ15.377\n\...,mutant ape yacht club bought floor h chg floor...,neu,neu,pos,pos\r
2,2022-12-30,24,0,Mutant Ape Yacht Club,🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#77 so...,mutant ape yacht club mayc nft sold eth k,neu,neu,pos,pos\r
3,2022-12-29,24,0,Mutant Ape Yacht Club,🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#23452...,mutant ape yacht club mayc nft sold eth k,neu,neu,pos,pos\r
4,2022-12-29,24,1,Mutant Ape Yacht Club,Mutant Ape Yacht Club #27908 sold for 17 ETH (...,mutant ape yacht club sold eth nft collection ...,neu,neu,pos,pos\r


In [20]:
df['label'] = df['label'].replace('neg',0)

In [21]:
df['label'] = df['label'].replace('pos',1)

In [22]:
df['label'] = df['label'].replace('neu',2)

In [23]:
df.head()

,Datetime,Quarter,Likes,NFT,Text,Clean Text,vader,textblob,label,Polarity (Reviewer 2)\r
0,2022-06-27,22,2,Mutant Ape Yacht Club,🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#27894...,mutant ape yacht club mayc nft sold eth k,neu,neu,1.0,pos\r
1,2023-02-11,31,2,Mutant Ape Yacht Club,🐳1 Mutant Ape Yacht Club bought for Ξ15.377\n\...,mutant ape yacht club bought floor h chg floor...,neu,neu,1.0,pos\r
2,2022-12-30,24,0,Mutant Ape Yacht Club,🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#77 so...,mutant ape yacht club mayc nft sold eth k,neu,neu,1.0,pos\r
3,2022-12-29,24,0,Mutant Ape Yacht Club,🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#23452...,mutant ape yacht club mayc nft sold eth k,neu,neu,1.0,pos\r
4,2022-12-29,24,1,Mutant Ape Yacht Club,Mutant Ape Yacht Club #27908 sold for 17 ETH (...,mutant ape yacht club sold eth nft collection ...,neu,neu,1.0,pos\r


In [24]:
df['label'].value_counts()

1.0    1053
2.0     453
0.0      95
Name: label, dtype: int64

In [25]:
df = df[['Clean Text','label']]

df.head()

,Clean Text,label
0,mutant ape yacht club mayc nft sold eth k,1.0
1,mutant ape yacht club bought floor h chg floor...,1.0
2,mutant ape yacht club mayc nft sold eth k,1.0
3,mutant ape yacht club mayc nft sold eth k,1.0
4,mutant ape yacht club sold eth nft collection ...,1.0


In [26]:
df.isnull().values.any()

True

In [27]:
df = df.dropna()

In [28]:
df.isnull().values.any()

False

In [29]:
X_train, X_test, y_train, y_test = train_test_split(df['Clean Text'], df['label'], test_size=0.2,random_state=42)

In [30]:
y_train.value_counts()

1.0    848
2.0    353
0.0     79
Name: label, dtype: int64

In [31]:
vectorizer = TfidfVectorizer(min_df = 1,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

In [32]:
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized=vectorizer.transform(X_test)

In [33]:
embed_aug = EmbeddingAugmenter()

In [34]:
augmented_sentences=[]
augmented_sentences_labels=[]
for i in X_train.index:
  if y_train[i]==0:
    temps3=embed_aug.augment(X_train[i])
    for sent in temps3:
      augmented_sentences.append(sent)
      augmented_sentences_labels.append(0)

In [35]:
X_train=X_train.append(pd.Series(augmented_sentences),ignore_index=True)
y_train=y_train.append(pd.Series(augmented_sentences_labels),ignore_index=True)

print(X_train.shape)
print(y_train.shape)

(1359,)
(1359,)


C:\Users\Alvin\AppData\Local\Temp\ipykernel_2468\31741503.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train=X_train.append(pd.Series(augmented_sentences),ignore_index=True)
C:\Users\Alvin\AppData\Local\Temp\ipykernel_2468\31741503.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_train=y_train.append(pd.Series(augmented_sentences_labels),ignore_index=True)


In [36]:
y_train.value_counts()

1.0    848
2.0    353
0.0    158
dtype: int64

In [37]:
aug = naw.SynonymAug(aug_src='wordnet',aug_max=3)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Alvin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Alvin\AppData\Roaming\nltk_data...


In [38]:
augmented_sentences=[]
augmented_sentences_labels=[]
for i in X_train.index:
  if y_train[i]==0:
    temps1=aug.augment(X_train[i],n=3)
    for sent in temps1:
      augmented_sentences.append(sent)
      augmented_sentences_labels.append(0)

In [39]:
X_train=X_train.append(pd.Series(augmented_sentences),ignore_index=True)
y_train=y_train.append(pd.Series(augmented_sentences_labels),ignore_index=True)

print(X_train.shape)
print(y_train.shape)

(1833,)
(1833,)


C:\Users\Alvin\AppData\Local\Temp\ipykernel_2468\31741503.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train=X_train.append(pd.Series(augmented_sentences),ignore_index=True)
C:\Users\Alvin\AppData\Local\Temp\ipykernel_2468\31741503.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_train=y_train.append(pd.Series(augmented_sentences_labels),ignore_index=True)


In [40]:
y_train.value_counts()

1.0    848
0.0    632
2.0    353
dtype: int64

In [41]:
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized=vectorizer.transform(X_test)

In [42]:
LGB_classifier = LGBMClassifier()

LGB_classifier.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': 'warn',
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [43]:
t0 = time.time()
LGB_classifier = LGBMClassifier(seed=0).fit(X_train_vectorized,y_train)
t1 = time.time()

time_linear_train = t1-t0

In [44]:
y_pred=LGB_classifier.predict(X_test_vectorized)

In [45]:
print("Training time: %fs" % (time_linear_train))

Training time: 560.315035s


In [46]:
print(accuracy_score(y_test, y_pred))

0.8193146417445483


In [47]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.50      0.50      0.50        16
         1.0       0.85      0.91      0.88       205
         2.0       0.81      0.69      0.75       100

    accuracy                           0.82       321
   macro avg       0.72      0.70      0.71       321
weighted avg       0.82      0.82      0.82       321



In [48]:
# Hyperparameter tuning
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV

In [49]:
# Define the search space
param_grid = { 
    "learning_rate": [0.1, 0.01],
    "max_depth": [3, 5, 7, 9],
    "num_leaves": [25, 50, 75, 100],
    "feature_fraction": [0.3, 0.5, 0.8],
    "bagging_fraction": [0.3, 0.5, 0.8],
    "min_child_samples": [10, 30, 50]
    }
# Set up score
scoring = ['f1']
# Set up the k-fold cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

In [50]:
# Define grid search
grid_search = GridSearchCV(estimator=LGB_classifier, 
                           param_grid=param_grid, 
                           scoring=scoring, 
                           refit='f1', 
                           n_jobs=-1, 
                           cv=kfold, 
                           verbose=0)
# Fit grid search
grid_result = grid_search.fit(X_train_vectorized,y_train)
# Print grid search summary
grid_result
# Print the best score and the corresponding hyperparameters
print(f'The best score is {grid_result.best_score_:.4f}')
#print('The best score standard deviation is', round(grid_result.cv_results_['std_test_recall'][grid_result.best_index_], 4))
print(f'The best hyperparameters are {grid_result.best_params_}')

[LightGBM] [Warning] feature_fraction is set=0.3, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3
[LightGBM] [Warning] bagging_fraction is set=0.3, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3
The best score is nan
The best hyperparameters are {'bagging_fraction': 0.3, 'feature_fraction': 0.3, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_samples': 10, 'num_leaves': 25}


C:\Users\Alvin\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan n

In [51]:
# Make prediction using the best model
grid_predict = grid_search.predict(X_test_vectorized)
# Get predicted probabilities
#grid_predict_prob = grid_search.predict_proba(X_test_vectorized)[:,1]

In [52]:
print(accuracy_score(y_test, y_pred))

0.8193146417445483


In [53]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.50      0.50      0.50        16
         1.0       0.85      0.91      0.88       205
         2.0       0.81      0.69      0.75       100

    accuracy                           0.82       321
   macro avg       0.72      0.70      0.71       321
weighted avg       0.82      0.82      0.82       321

